In [1]:
import numpy as np
import pandas as pd

On se connecte à la base de données

In [2]:
from db_utils import get_db_engine

# Obtenir le moteur de base de données
engine = get_db_engine()

In [3]:
# Charger les fichiers sources
file1 = 'data\\ObesityDataSet_raw_and_data_sinthetic.csv'
file2 = 'data\\lifestyle_sustainability_data.csv'
file3 = 'data\\Sleep_health_and_lifestyle_dataset.csv'
file4 = 'data\\DietSurvey_toJson.json'

# Charger les fichiers dans des DataFrames
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_json(file4)

# Ajouter des IDs uniques pour chaque fichier
df1['id_person'] = range(1, len(df1) + 1)
df2['id_person'] = range(len(df1) + 1, len(df1) + len(df2) + 1)
df3['id_person'] = range(len(df1) + len(df2) + 1, len(df1) + len(df2) + len(df3) + 1)
df4['id_person'] = range(len(df1) + len(df2) + len(df3) + 1, len(df1) + len(df2) + len(df3) + len(df4) + 1)


In [4]:
# drinks1 - df1
drinks1 = df1[['SCC', 'CH2O']].rename(
    columns={
        'SCC': 'sugary_con',
        'CH2O': 'water_con'
    }
)
drinks1['alcohol_con'] = None  # Pas de colonne correspondante dans df1

print("drinks1 préparé.")

# drinks4 - df4
drinks4 = df4[
    [
        "What is your water consumption like (in a day, 1 cup=250ml approx)",
        "How frequently do you consume these beverages [Aerated (Soft) Drinks]",
        "How frequently do you consume these beverages [Alcoholic Beverages]"
    ]
].rename(
    columns={
        "What is your water consumption like (in a day, 1 cup=250ml approx)": "water_con",
        "How frequently do you consume these beverages [Alcoholic Beverages]": "alcohol_con",
        "How frequently do you consume these beverages [Aerated (Soft) Drinks]": "sugary_con"
    }
)

print("drinks4 préparé.")


drinks1 préparé.
drinks4 préparé.


In [5]:
# Fonction pour convertir la note en catégorie 'water_con'
def categorize_water_con(note):
    if note >= 2.7:
        return 'More than 15 cups'
    elif note >= 2.3:
        return '11-14 cups'
    elif note >= 1.7:
        return '7-10 cups'
    elif note >= 1.0:
        return '4-6 cups'
    else:
        return 'Less than 3 cups'

# Appliquer la fonction à la colonne 'water_con' de food_habits1
drinks1 ['water_con'] = drinks1 ['water_con'].apply(categorize_water_con)

In [6]:
import numpy as np

# Définir les options pour "yes"
yes_options = ['Once a month', 'Few times a week', 'Less often']

# Appliquer la transformation
drinks1['sugary_con'] = drinks1['sugary_con'].apply(
    lambda x: 'Never' if x == 'no' else np.random.choice(yes_options)
)

drinks1

,sugary_con,water_con,alcohol_con
0,Never,7-10 cups,None
1,Less often,More than 15 cups,None
2,Never,7-10 cups,None
3,Never,7-10 cups,None
4,Never,7-10 cups,None
...,...,...,...
2106,Never,7-10 cups,None
2107,Never,7-10 cups,None
2108,Never,7-10 cups,None
2109,Never,More than 15 cups,None


In [7]:
# Fonction pour générer un nombre aléatoire dans les plages de chaque catégorie
def randomize_water_con(value):
    if value == 'More than 15 cups':
        return np.random.randint(15, 25)  # Entre 15 et 20
    elif value == '11-14 cups':
        return np.random.randint(11, 14)  # Entre 11 et 14
    elif value == '7-10 cups':
        return np.random.randint(7, 10)   # Entre 7 et 10
    elif value == '4-6 cups':
        return np.random.randint(4, 6)    # Entre 4 et 6
    elif value == 'Less than 3 cups':
        return np.random.randint(0, 3)    # Entre 0 et 3
    else:
        return value

# Appliquer la fonction à la colonne 'water_con'
drinks1['water_con'] = drinks1['water_con'].apply(randomize_water_con)
drinks4['water_con'] = drinks4['water_con'].apply(randomize_water_con)


In [8]:
# Concatenation des DataFrames préparés
drinks = pd.concat([drinks1, drinks4], ignore_index=True)
print("Drinks combiné.")

Drinks combiné.


In [9]:
# Obtenir les lignes uniques
unique_rows = drinks.drop_duplicates()

# Afficher les lignes uniques
unique_rows.head()


,sugary_con,water_con,alcohol_con
0,Never,7,None
1,Less often,23,None
3,Never,8,None
5,Never,9,None
10,Never,20,None


In [10]:
# Supprimer les doublons pour conserver uniquement les lignes uniques
drinks_unique = drinks.drop_duplicates().reset_index(drop=True)

# Ajouter la colonne 'id_drinks' avec des valeurs incrémentées
drinks_unique.insert(0, 'id_drinks', range(1, len(drinks_unique) + 1))

# Afficher le DataFrame modifié
drinks_unique


,id_drinks,sugary_con,water_con,alcohol_con
0,1,Never,7,None
1,2,Less often,23,None
2,3,Never,8,None
3,4,Never,9,None
4,5,Never,20,None
...,...,...,...,...
91,92,Once a day,8,Never
92,93,Never,7,Once a month
93,94,Once a day,9,Few times a week
94,95,Never,20,Few times a week


In [11]:
drinks_unique.to_sql('Drinks', engine, if_exists='append', index=False)

print("Les données ont été insérées avec succès dans la table 'Diet'.")


Les données ont été insérées avec succès dans la table 'Diet'.


C:\Users\user\AppData\Local\Temp\ipykernel_25112\3162262561.py:1: UserWarning: The provided table name 'Drinks' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  drinks_unique.to_sql('Drinks', engine, if_exists='append', index=False)


In [12]:
# Ajouter l'id_person pour chaque DataFrame avant la concaténation
drinks1['id_person'] = df1['id_person']
drinks4['id_person'] = df4['id_person']

# Concatenation des DataFrames préparés
id_and_drinks = pd.concat([drinks1, drinks4], ignore_index=True)

id_and_drinks

,sugary_con,water_con,alcohol_con,id_person
0,Never,7,None,1
1,Less often,23,None,2
2,Never,7,None,3
3,Never,8,None,4
4,Never,7,None,5
...,...,...,...,...
2184,Once a day,9,Few times a week,3058
2185,Never,20,Few times a week,3059
2186,Once a month,9,Never,3060
2187,Once a month,4,Few times a week,3061


In [16]:
# Fusionner id_and_drinks avec drinks_unique sur les colonnes correspondantes
id_and_drinks_with_ids = pd.merge(
    id_and_drinks[['id_person', 'sugary_con', 'water_con', 'alcohol_con']],  # Colonnes à utiliser dans id_and_drinks
    drinks_unique[['sugary_con', 'water_con', 'alcohol_con', 'id_drinks']],  # Colonnes à utiliser dans drinks_unique
    on=['sugary_con', 'water_con', 'alcohol_con'],  # Clés de correspondance
    how='left'  # Type de jointure, ici 'left' pour garder toutes les lignes de id_and_drinks
)

# Sélectionner seulement les colonnes 'id_person' et 'id_drinks'
id_and_drinks_final = id_and_drinks_with_ids[['id_person', 'id_drinks']]

# Afficher le résultat final
id_and_drinks_final


,id_person,id_drinks
0,1,1
1,2,2
2,3,1
3,4,3
4,5,1
...,...,...
2184,3058,94
2185,3059,95
2186,3060,74
2187,3061,96


In [17]:
# Ajouter les id_person de file2 et file3 avec id_drinks = 97
# Créer un DataFrame pour file2
file2_drinks = pd.DataFrame({
    'id_person': df2['id_person'],
    'id_drinks': [97] * len(df2)  # id_drinks = 97 pour toutes les lignes
})

# Créer un DataFrame pour file3
file3_drinks = pd.DataFrame({
    'id_person': df3['id_person'],
    'id_drinks': [97] * len(df3)  # id_drinks = 97 pour toutes les lignes
})

# Concaténer les DataFrames id_and_drinks, file2_drinks et file3_drinks
final_df = pd.concat([id_and_drinks_final, file2_drinks, file3_drinks], ignore_index=True)

# Trier par id_person pour que les id_person soient dans l'ordre croissant
final_df = final_df.sort_values(by='id_person').reset_index(drop=True)

# Afficher le DataFrame final avec les id_person dans l'ordre croissant
final_df



,id_person,id_drinks
0,1,1
1,2,2
2,3,1
3,4,3
4,5,1
...,...,...
3057,3058,94
3058,3059,95
3059,3060,74
3060,3061,96
